In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
data= pd.read_excel('/content/drive/MyDrive/Tanjila_mam/DS-Healthcare_version_2.xlsx')
result = {}
target = 'Type'

In [3]:
data[target].value_counts()

ischemic      581219
healthy       267360
arrhythmic     40238
Name: Type, dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

data[target]=encoder.fit_transform(data[target])

In [5]:
X= data.loc[:,data.columns != target]
y = data[target]

In [6]:
def fun_split(X,Y,test_size,feature):

    f=Y.value_counts()
    rows=X.shape[0]
    talika=[]
    count=0
    for i in f:
        talika.append(i)
    train_size=1-test_size
    start=0

    switch=False

    for i in talika :

        train=start+int(i*train_size)
        f= X[feature][train]
        count1=0
        count2=0
        index1=train
        index2=train+1
        end=start+i-1

        while X[feature][index1]==f:
            if index1<start:
                break
            index1-=1
            count1+=1


        while X[feature][index2]==f:
            if index2>end:
                break
            index2+=1
            count2+=1

        if count1>=count2:
            train+=count2
        else:
            train-=count1


        if switch == False:
            X_train= X[start:train+1]
            X_test= X[train+1:end]
            Y_train = Y[start:train+1]
            Y_test = Y[train+1:end]
            switch=True

        else:
            X_train=pd.concat([X_train, X[start:train+1]], ignore_index=True)
            X_test=pd.concat([X_test, X[train+1:end+1]], ignore_index=True)
            Y_train=pd.concat([Y_train, Y[start:train+1]], ignore_index=True)
            Y_test=pd.concat([Y_test, Y[train+1:end+1]], ignore_index=True)

        start=start+i

    print('Train Percentage:',X_train.shape[0]/rows)

    return X_train,X_test,Y_train,Y_test

In [7]:
 X_train, X_test, y_train, y_test = fun_split(X,y, test_size=0.2, feature= 'Subject')

Train Percentage: 0.8002839729663137


In [8]:
X_train= X_train.loc[:,X_train.columns != 'Subject']
X_train= X_train.loc[:,X_train.columns != 'Time']
X_train= X_train.loc[:,X_train.columns != 'Cycle']

X_test= X_test.loc[:,X_test.columns != 'Subject']
X_test= X_test.loc[:,X_test.columns != 'Time']
X_test= X_test.loc[:,X_test.columns != 'Cycle']

#GradientBoost

In [ ]:

from sklearn.ensemble import GradientBoostingClassifier

gradBoost_depth = GradientBoostingClassifier(max_depth=1,random_state=0)
gradBoost_depth.fit(X_train, y_train)
y_pred = gradBoost_depth.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(gradBoost_depth,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

[[   902    821   5836]
 [  1005  40958  11172]
 [   448     13 116355]]
              precision    recall  f1-score   support

           0       0.38      0.12      0.18      7559
           1       0.98      0.77      0.86     53135
           2       0.87      1.00      0.93    116816

    accuracy                           0.89    177510
   macro avg       0.75      0.63      0.66    177510
weighted avg       0.88      0.89      0.88    177510

Accurecy:  0.8913018984845924


#RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_all = RandomForestClassifier(n_estimators=2,max_depth=8,random_state=0)
rf_all.fit(X_train, y_train)
y_pred=rf_all.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_all,3,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

[[   693    963   5903]
 [  5235  32101  15799]
 [  1016     57 115743]]
              precision    recall  f1-score   support

           0       0.10      0.09      0.10      7559
           1       0.97      0.60      0.74     53135
           2       0.84      0.99      0.91    116816

    accuracy                           0.84    177510
   macro avg       0.64      0.56      0.58    177510
weighted avg       0.85      0.84      0.83    177510

Accurecy:  0.8367810264210467


#XGBoost

In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xgboost as xgb
xgb_depth = xgb.XGBClassifier(max_depth=1,random_state=0)
xgb_depth.fit(X_train.values,y_train.values)
y_pred = xgb_depth.predict(X_test.values)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_depth,4,'xgboost')]=accuracy_score(y_test, y_pred)
print(xgb_depth)

[[  1554    896   5109]
 [  2382  40560  10193]
 [   987     15 115814]]
              precision    recall  f1-score   support

           0       0.32      0.21      0.25      7559
           1       0.98      0.76      0.86     53135
           2       0.88      0.99      0.93    116816

    accuracy                           0.89    177510
   macro avg       0.73      0.65      0.68    177510
weighted avg       0.89      0.89      0.88    177510

Accurecy:  0.8896850881640471
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=Non

#Histogram-Based Gradient Boosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score

hisgradBoost_max_depth = HistGradientBoostingClassifier(max_depth=1,random_state=0)
hisgradBoost_max_depth.fit(X_train, y_train)
y_pred = hisgradBoost_max_depth.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(hisgradBoost_max_depth,5,'HistGradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

[[  1177    899   5483]
 [  1560  42812   8763]
 [   865     17 115934]]
              precision    recall  f1-score   support

           0       0.33      0.16      0.21      7559
           1       0.98      0.81      0.88     53135
           2       0.89      0.99      0.94    116816

    accuracy                           0.90    177510
   macro avg       0.73      0.65      0.68    177510
weighted avg       0.89      0.90      0.89    177510

Accurecy:  0.9009238916117401


#Model select

In [ ]:

model = hisgradBoost_max_depth
#model = xgb_depth
#model = rf_all
#model = gradBoost_depth

#Confusion Matrix

In [ ]:
def confusion_details(y_test,y_pred):
    labels = list(set(y_test))
    labels.sort()

    print("Total labels: %s -> %s" % (len(labels), labels))

    df = pd.DataFrame(
        data=confusion_matrix(y_test, y_pred, labels=labels),
        columns=labels,
        index=labels
    )

    print(df)

    print()
    print("----------------------------------------------------------------------------------------")
    print("----------------------------------------------------------------------------------------")
    print()
    #
    # Local (metrics per class)
    #
    tps = {}
    fps = {}
    fns = {}
    tns = {}

    precision_local = {}
    recall_local = {}
    f1_local = {}
    accuracy_local = {}
    specificity_local={}

    for label in labels:
        tps[label] = df.loc[label, label]
        fps[label] = df[label].sum() - tps[label]
        fns[label] = df.loc[label].sum() - tps[label]
        tns[label]=len(y_test) - (tps[label] + fps[label] + fns[label])

        tp, fp, fn, tn = tps[label], fps[label], fns[label], tns[label]

        precision_local[label] = tp / (tp + fp) if (tp + fp) > 0. else 0.
        specificity_local[label] = tn / (tn + fp) if (tn + fp) > 0. else 0.
        recall_local[label] = tp / (tp + fn) if (tp + fp) > 0. else 0.
        p, r = precision_local[label], recall_local[label]

        f1_local[label] = 2. * p * r / (p + r) if (p + r) > 0. else 0.
        accuracy_local[label] = tp / (tp + fp + fn) if (tp + fp + fn) > 0. else 0.



    print("#-- Local measures --#")
    print("True Positives(TP):", tps)
    print("False Positives(FP):", fps)
    print("True Negatives(TN):", tns)
    print("False Negatives(FN):", fns)
    print("----------------------------")

    print("Precision:", precision_local)
    print("Recall/Sensitivity:", recall_local)
    print("Specificity:",specificity_local)
    print("F1-Score:", f1_local)
    print("Accuracy:", accuracy_local)


    print()
    print("----------------------------------------------------------------------------------------")
    print("----------------------------------------------------------------------------------------")
    print()
    #
    # Global
    #
    micro_averages = {}
    macro_averages = {}

    correct_predictions = sum(tps.values())
    true_negative=sum(tns.values())

    den = sum(list(tps.values()) + list(fps.values()))
    micro_averages["Precision"] = 1. * correct_predictions / den if den > 0. else 0.

    den = sum(list(tps.values()) + list(fns.values()))
    micro_averages["Recall"] = 1. * correct_predictions / den if den > 0. else 0.

    den = sum(list(tns.values()) + list(fps.values()))
    micro_averages["Specificity"] = 1. * true_negative / den if den > 0. else 0.


    micro_avg_p, micro_avg_r = micro_averages["Precision"], micro_averages["Recall"]
    micro_averages["F1-score"] = 2. * micro_avg_p * micro_avg_r / (micro_avg_p + micro_avg_r) if (micro_avg_p + micro_avg_r) > 0. else 0.

    macro_averages["Precision"] = np.mean(list(precision_local.values()))
    macro_averages["Recall"] = np.mean(list(recall_local.values()))
    macro_averages["Specificity"]=np.mean(list(specificity_local.values()))


    macro_avg_p, macro_avg_r = macro_averages["Precision"], macro_averages["Recall"]
    macro_averages["F1-Score"] = 2. * macro_avg_p * macro_avg_r / (macro_avg_p + macro_avg_r) if (macro_avg_p + macro_avg_r) > 0. else 0.

    total_predictions = df.values.sum()
    accuracy_global = correct_predictions / total_predictions if total_predictions > 0. else 0.

    print("#-- Global measures --#")
    print("Micro-Averages:", micro_averages)
    print("Macro-Averages:", macro_averages)
    print("Correct predictions:", correct_predictions)
    print("Total predictions:", total_predictions)
    print("Accuracy:", accuracy_global)


    print()
    print("----------------------------------------------------------------------------------------")
    print("----------------------------------------------------------------------------------------")
    print()



    accuracy_local_new = {}
    for label in labels:
        tp, fp, fn, tn = tps[label], fps[label], fns[label], tns[label]
        accuracy_local_new[label] = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0. else 0.

    total_true = sum(list(tps.values()) + list(tns.values()))
    total_predictions = sum(list(tps.values()) + list(tns.values()) + list(fps.values()) + list(fns.values()))
    accuracy_global_new = 1. * total_true / total_predictions if total_predictions > 0. else 0.

    print("Accuracy (per class), with TNs:", accuracy_local_new)
    print("Accuracy (per class), without TNs:", accuracy_local)
    print("Accuracy (global), with TNs:", accuracy_global_new)
    print("Accuracy (global), without TNs:", accuracy_global)

In [ ]:
#model = hisgradBoost_max_depth
#model = xgb_depth
model = rf_all
#model = gradBoost_depth

In [ ]:
y_pred = model.predict(X_test)
confusion_details(y_test,y_pred)

Total labels: 3 -> [0, 1, 2]
      0      1       2
0   693    963    5903
1  5235  32101   15799
2  1016     57  115743

----------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------

#-- Local measures --#
True Positives(TP): {0: 693, 1: 32101, 2: 115743}
False Positives(FP): {0: 6251, 1: 1020, 2: 21702}
True Negatives(TN): {0: 163700, 1: 123355, 2: 38992}
False Negatives(FN): {0: 6866, 1: 21034, 2: 1073}
----------------------------
Precision: {0: 0.09979838709677419, 1: 0.9692038283868241, 2: 0.8421041143730219}
Recall/Sensitivity: {0: 0.09167879349120255, 1: 0.6041403971017221, 2: 0.9908146144363785}
Specificity: {0: 0.963218810127625, 1: 0.9917989949748743, 2: 0.6424358256170297}
F1-Score: {0: 0.09556643453078673, 1: 0.7443192357633093, 2: 0.9104266875376089}
Accuracy: {0: 0.050181028240405504, 1: 0.5927615178653864, 2: 0.8355809353297045}

-------------------------

In [ ]:
import matplotlib.pyplot as plt
from yellowbrick.classifier import ConfusionMatrix
import matplotlib.dates as dates



def Conf_Mat(X_,y_,attb,model,cls):


  fig = plt.figure(figsize=(attb["Fig Height"],attb["Fig weidth"]))
  ax = fig.add_subplot(111)

  cm = ConfusionMatrix(model, classes=cls,fontsize=attb["Confusion Matrix Inner Fontsize"],ax=ax)
  cm.fit(X_train, y_train)
  cm.score(X_, y_)

  cm.ax.set_xlabel("Predicted Class", fontsize=attb["X Axis Fontsize"],fontweight="bold")
  cm.ax.set_ylabel("True Class", fontsize=attb["Y Axis Fontsize"],fontweight="bold")
  cm.ax.xaxis.set_tick_params(labelsize=attb["X Label Fontsize"])
  cm.ax.yaxis.set_tick_params(labelsize=attb["Y Label Fontsize"])
  for label in ax.get_xticklabels():
    label.set_fontweight(550)
  for label in ax.get_yticklabels():
    label.set_fontweight(550)

  plt.savefig("Testing_confusion_mat"+str(model)+".png")
  cm.show()
  print("--------------------")
  print("--------------------")

In [ ]:
attributes={
   "Fig Height": 10,
   "Fig weidth": 10,

  "Y Axis Fontsize" : 20,
  "X Axis Fontsize" : 20,

  "Y Label Fontsize" : 20,
  "X Label Fontsize" : 20,

  "Confusion Matrix Inner Fontsize": 18


}

Conf_Mat(X_test,y_test,attributes,classifier,labels)

#ROC CURV

In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.4 MB/s eta 0:00:00


In [ ]:
lebel_dict={
    0:"arrhythmic",
    1:"ischemic",
    2:"healthy"

}

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt

def ploty_ROC_1(model,lebel_dict):

  y_scores = model.predict_proba(X_test)

  y_onehot = pd.get_dummies(y_test, columns=model.classes_)

  fig = go.Figure()
  fig.add_shape(
      type='line', line=dict(dash='dash'),
      x0=0, x1=1, y0=0, y1=1
  )

  for i in range(y_scores.shape[1]):
      y_true = y_onehot.iloc[:, i]
      y_score = y_scores[:, i]

      fpr, tpr, _ = roc_curve(y_true, y_score)
      auc_score = roc_auc_score(y_true, y_score)
      name = f"<b><i>{lebel_dict[y_onehot.columns[i]]} (AUC={auc_score:.2f})<i></b>"
      fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'))

  fig.update_layout(
      xaxis_title='<b>False Positive Rate</b>',
      yaxis_title='<b>True Positive Rate</b>',
      yaxis=dict(scaleanchor="x", scaleratio=1,griddash='dot', minor_griddash="dot",gridcolor='Black',zerolinewidth=4, zerolinecolor='LightPink'),
      xaxis=dict(constrain='domain',griddash='dot', minor_griddash="dot",gridcolor='Black',zerolinewidth=4, zerolinecolor='LightPink'),
      width=1000, height=1000,
      font=dict(
        family="Courier New, monospace",
        size=20,
        color="BLack"
    ),
    legend=dict(
        x=0.582,
        y=0.07,
        traceorder="reversed",
        title_font_family="Times New Roman",
        font=dict(
            family="Courier New, monospace",
            size=20,
            color="black"
        ),
        bgcolor="lightblue",
        bordercolor="Black",
        borderwidth=2
    )
  )
  fig.show()
  fig.write_image(str(model)[:5]+"_roc.png")


#ploty_ROC_1(model,lebel_dict)

NameError: ignored

#Cross Validation

In [ ]:
"""from sklearn.model_selection import cross_val_score
X_= X.loc[:,X.columns != 'Subject']
X_= X_.loc[:,X_.columns != 'Time']
X_= X_.loc[:,X_.columns != 'Cycle']
k=5
#for i in result:
#print(i[0]," -> Accuracy: ",)
l=list(cross_val_score(model,X_, y,cv=k))
avg=sum(l)/k
print(model," -> AVG Accurecy After CV: "+str(avg)+ " (For "+str(k)+" Fold)")
print("--------------------------------------------------------------------------")"""

#SHAP

In [ ]:
!pip install shap
import shap

In [ ]:
def SHAP_EXP(model,graph_feat,list_of_feat1,start_index,end_index):
  print("Models: ",model)

  explainer = shap.Explainer(model.predict, X_test)

  shap_values1 = explainer(X_test[start_index:end_index])
  features_names=list_of_feat1


  print("Global Explaination")
  shap.plots.bar(shap_values1,max_display=graph_feat["max_display"])

  print("Global Explaination")

  shap.summary_plot(shap_values1,max_display=graph_feat["max_display"])

  print("---------------------")

  print("Local Explaination")
  shap.plots.waterfall(shap_values1[graph_feat["shap_values Index"]],max_display=graph_feat["max_display"])


  print("Local Explaination")

  shap.plots.bar(shap_values1[graph_feat["shap_values Index"]],max_display=graph_feat["max_display"])


In [ ]:
graph_feat={
    "max_display":30,
    "shap_values Index":10
}
start_index,end_index=0,100

SHAP_EXP(model,graph_feat,list(X_test.columns),start_index,end_index)

#Lime

In [ ]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283839 sha256=d4e1598832aaba3df678669222d1c8e18ecfa3e38fe5cd77bfa5f71ff9f45af9
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
import lime
from lime import lime_tabular

def LIME_EXP(model,row,num_features,top_labels):

    if str(model)[:3] == "XGB":

      explainer = lime.lime_tabular.LimeTabularExplainer(
        X_train.values,
        feature_names=list(list(X_train.columns)),
        class_names=['arrhythmic', 'healthy', 'ischemic'],
        mode='classification'
        )

      exp = explainer.explain_instance(X_test.iloc[row, :].values,
                                 model.predict_proba,
                                 num_features=num_features,
                                 top_labels=top_labels)




    else:
      explainer = lime_tabular.LimeTabularExplainer(
        training_data=np.array(X_train),
        feature_names=list(X_train.columns),
        class_names=['arrhythmic', 'healthy', 'ischemic'],
        mode='classification'
        )

      exp = explainer.explain_instance(X_test.iloc[row],
                                      model.predict_proba,
                                      num_features=num_features,
                                      top_labels=top_labels)



    exp.show_in_notebook(show_table=True, show_all=True)



    from IPython.display import HTML
    html_data = exp.as_html()
    HTML(data=html_data)

    exp.save_to_file(str(model)[0:10]+".html")

In [ ]:

model = hisgradBoost_max_depth
#model = xgb_depth
#model = rf_all
#model = gradBoost_depth

In [ ]:
row = 7#43#int(input("Enter the index of row to explain: "))
num_features,top_labels = 6,4

print("Model:",hisgradBoost_max_depth)
LIME_EXP(hisgradBoost_max_depth,row,num_features,top_labels)

In [ ]:
#row = #int(input("Enter the index of row to explain: "))


from tqdm.notebook import tqdm

for row in tqdm(range(0,200)):
  print()
  print()
  print("--------------------------------------------------")
  print("--------------------------------------------------")
  print("Row Number: ", row)
  print("----------")
  num_features,top_labels = 6,4

  print("Model:",hisgradBoost_max_depth)
  LIME_EXP(hisgradBoost_max_depth,row,num_features,top_labels)
  print()
  print("Model:",xgb_depth)
  LIME_EXP(xgb_depth,row,num_features,top_labels)
  print()
  print("Model:",rf_all)
  LIME_EXP(rf_all,row,num_features,top_labels)
  print()
  print("Model:",gradBoost_depth)
  LIME_EXP(gradBoost_depth,row,num_features,top_labels)

  print("--------------------------------------------------")
  print("--------------------------------------------------")
  print("--------------------------------------------------")

Output hidden; open in https://colab.research.google.com to view.